<h1> Cluster Analysis of Toronto City </h1>

In [1]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h4> Loading the data created after step 2 </h4>

In [3]:
df_toronto = pd.read_csv('Toronto_step2.csv')
df_toronto.head(5)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,3,M1C,Scarborough,Port Union,43.784535,-79.160497
4,4,M1C,Scarborough,Rouge Hill,43.784535,-79.160497


<h3> Create a map of Toronto </h3>

In [4]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

<h4> Create a new data frame with neighborhoods in Scarborough </h4>

In [5]:
#@hiddel_cell
CLIENT_ID = 'URBE4S4KZ3JZLFYNGR32I5AM4MO31IT1RKH3O3DID51EIWSD' # your Foursquare ID
CLIENT_SECRET = '3EL2E5QDYRA34LJ15UJCE3OSCP0XIAOBJAO3RRLV5ZNG5XWN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,3,M1C,Scarborough,Port Union,43.784535,-79.160497
4,4,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
5,5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h4> Create a map of Scarborough and its neighbourhoods </h4>

In [7]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [8]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

<h4>Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough </h4>

In [10]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge" are 43.8066863, -79.19435340000003.


In [11]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4744ef4434b947c55adca7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLa

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)


,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
2,DAVIDsTEA,Tea Room,43.776613,-79.258516
3,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
7,Scarborough Town Centre,Shopping Mall,43.775545,-79.257545
8,Moxie's Classic Grill,American Restaurant,43.776219,-79.256820
9,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4> Get venues for each neighborhood in Scarborough </h4>

In [17]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge
Malvern
Highland Creek
Port Union
Rouge Hill
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park
Ionview
East Birchmount Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Cliffside West
Birch Cliff
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Tam O'Shanter
Sullivan
Clarks Corners
Steeles East
Milliken
L'Amoreaux East
Agincourt North
L'Amoreaux West
Steeles West
Upper Rouge


In [18]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Highland Creek,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
5,Port Union,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
6,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
7,Rouge Hill,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
8,Guildwood,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
9,Guildwood,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [19]:
scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
193,Steeles West,43.799525,-79.318389,Subway,43.798983,-79.318838,Sandwich Place
194,Steeles West,43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop
195,Steeles West,43.799525,-79.318389,Shoppers Drug Mart,43.799670,-79.319315,Pharmacy
196,Steeles West,43.799525,-79.318389,KFC,43.798938,-79.318854,Fast Food Restaurant
197,Steeles West,43.799525,-79.318389,Yamamoto Japanese Cuisine 山本盛世,43.798589,-79.318558,Japanese Restaurant
198,Steeles West,43.799525,-79.318389,McDonald's,43.798880,-79.318724,Fast Food Restaurant
199,Steeles West,43.799525,-79.318389,Pizza Pizza,43.797909,-79.318113,Pizza Place
200,Steeles West,43.799525,-79.318389,Nails for You,43.798750,-79.318768,Cosmetics Shop
201,Steeles West,43.799525,-79.318389,A Buck or Two,43.798286,-79.318485,Thrift / Vintage Store
202,Steeles West,43.799525,-79.318389,Eggsmart,43.796375,-79.318681,Breakfast Spot


In [20]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clairlea,10,10,10,10,10,10
Clarks Corners,9,9,9,9,9,9
Cliffcrest,3,3,3,3,3,3
Cliffside,3,3,3,3,3,3
Cliffside West,4,4,4,4,4,4


In [21]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 53 uniques categories.


In [22]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Highland Creek,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Highland Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Port Union,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
scarb_onehot.shape

(203, 54)

In [24]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.0,0.000,0.000,0.0,0.25,0.0,0.0,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.250,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0
1,Agincourt North,0.000000,0.000,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.5,0.0,0.000000,0.000000,0.5,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
2,Birch Cliff,0.000000,0.000,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.25,0.000,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0
3,Cedarbrae,0.000000,0.125,0.0,0.125,0.125,0.0,0.00,0.0,0.0,0.00,0.125,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.125000,0.0,0.00,0.0,0.125,0.0,0.000000,0.000000,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.125000,0.0,0.0,0.0
4,Clairlea,0.000000,0.000,0.0,0.200,0.000,0.0,0.00,0.2,0.1,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.100000,0.000000,0.0,0.00,0.0,0.000,0.0,0.100000,0.000000,0.0,0.0,0.0,0.000,0.0,0.1,0.0,0.000000,0.0,0.000000,0.1,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.1,0.000000,0.0,0.0,0.0
5,Clarks Corners,0.000000,0.000,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.00,0.000,0.111111,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.111111,0.111111,0.0,0.00,0.0,0.000,0.0,0.000000,0.111111,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.111111,0.0,0.0,0.111111,0.222222,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.111111,0.0,0.0,0.0
6,Cliffcrest,0.333333,0.000,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000,0.0,0.333333,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0


<h3> Get top 10 venues per neighbourhood </h3> 

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
1,Agincourt North,Park,Playground,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Birch Cliff,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Cosmetics Shop
4,Clairlea,Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Bar,Cosmetics Shop
5,Clarks Corners,Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Pharmacy,College Stadium,Construction & Landscaping
6,Cliffcrest,Intersection,Motel,American Restaurant,Thai Restaurant,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,Cliffside,Intersection,Motel,American Restaurant,Thai Restaurant,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,Cliffside West,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,Dorset Park,Indian Restaurant,Chinese Restaurant,Furniture / Home Store,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Bar,Breakfast Spot,Grocery Store,General Entertainment


<h3> Run k-means to cluster the neighborhoods into 5 clusters</h3>

In [28]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([2, 0, 2, 2, 2, 2, 4, 4, 2, 2], dtype=int32)

<h3> Include kmeans.labels_ into the original Scarborough dataframe </h3> 

In [29]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,Rouge,43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,1,M1B,Scarborough,Malvern,43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
2,2,M1C,Scarborough,Highland Creek,43.784535,-79.160497,2,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,3,M1C,Scarborough,Port Union,43.784535,-79.160497,2,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,4,M1C,Scarborough,Rouge Hill,43.784535,-79.160497,2,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,5,M1E,Scarborough,Guildwood,43.763573,-79.188711,2,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Furniture / Home Store,Fried Chicken Joint
6,6,M1E,Scarborough,Morningside,43.763573,-79.188711,4,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Furniture / Home Store,Fried Chicken Joint
7,7,M1E,Scarborough,West Hill,43.763573,-79.188711,4,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Furniture / Home Store,Fried Chicken Joint
8,8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Cosmetics Shop


<h3> Visualize the clusters in the map </h3>

In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3> Examine each of the five clusters </h3> 

In [31]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1B,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
17,M1M,-79.239476,0,Intersection,Motel,American Restaurant,Thai Restaurant,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
21,M1N,-79.264848,0,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
28,M1T,-79.304302,0,Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Pharmacy,College Stadium,Construction & Landscaping
30,M1T,-79.304302,0,Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Pharmacy,College Stadium,Construction & Landscaping


In [32]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M1M,-79.239476,1,Intersection,Motel,American Restaurant,Thai Restaurant,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
25,M1R,-79.295849,1,Sandwich Place,Auto Garage,Bakery,Smoke Shop,Shopping Mall,Breakfast Spot,Fast Food Restaurant,Cosmetics Shop,Department Store,Discount Store


In [33]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
2,M1C,-79.160497,2,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,M1C,-79.160497,2,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1C,-79.160497,2,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,M1E,-79.188711,2,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Furniture / Home Store,Fried Chicken Joint
8,M1G,-79.216917,2,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1H,-79.239476,2,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Cosmetics Shop
10,M1J,-79.239476,2,Playground,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,M1K,-79.262029,2,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Train Station,Bank,Bar,Hakka Restaurant,Grocery Store,General Entertainment
12,M1K,-79.262029,2,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Train Station,Bank,Bar,Hakka Restaurant,Grocery Store,General Entertainment


In [34]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M1K,-79.262029,3,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Train Station,Bank,Bar,Hakka Restaurant,Grocery Store,General Entertainment
24,M1P,-79.273304,3,Indian Restaurant,Chinese Restaurant,Furniture / Home Store,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Bar,Breakfast Spot,Grocery Store,General Entertainment
26,M1R,-79.295849,3,Sandwich Place,Auto Garage,Bakery,Smoke Shop,Shopping Mall,Breakfast Spot,Fast Food Restaurant,Cosmetics Shop,Department Store,Discount Store


In [35]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1E,-79.188711,4,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Furniture / Home Store,Fried Chicken Joint
7,M1E,-79.188711,4,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Furniture / Home Store,Fried Chicken Joint
29,M1T,-79.304302,4,Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Pharmacy,College Stadium,Construction & Landscaping
